## Reproduction project: Estimating individual treatment effect
Julia Wilbers (4470249) J.E.Wilbers@student.tudelft.nl
Juan Molano (5239540) J.E.MolanoValencia@student.tudelft.nl

## Introduction
The goal of this project was to recreate the neural network, and the results (partly) of the following paper: Estimating individual treatment effect: generalization bounds and algorithms (Uri Shalit et al.). [1] The authors of this paper aim to predict the individual treatment effect (ITE) from observational data. Therefore, the authors proposed a CFR (Counterfactual Regression) framework. In addition, the authors used a Tensorflow framework for the creating of their network. In this project we aim to reconstruct their network in pytorch.

It should be noted that the authors of the original paper designed the code in such way that a lot of options are adjustable. For example different types of loss functions, regularisation methods, number of layers etc. In this project we choose to not experiment with all these different settings and mostly use the settings as defined in code (default settings). The only setting from the default configuration that we changed was the choice of the imbalance measure (see section: Correction for distribution difference).

## Individual treatment effect
The individual treatment effect (ITE) can be seen as the effect of a certain treatment for a specific patient. One can imagine that it is very useful to estimate the outcome of a given treatment before the patient receive the treatment.A doctor could then choose the treatment based on the estimated outcome. Observational data e.g. given medication is used order to make such predictions. The ITE can be calculated using the following formula:

$ITE (x) = E[Y_1 - Y_0 | x]$

Where $Y_1$ is the outcome given the treatment and $Y_0$ the outcome not given the treatment. Then the ITE of a certain treatment is then the expected outcome given the treatment minus the expected outcome not given the treatment when observing data x.

## IHDP dataset
This project makes use of the IHDP (Infant Health and Development Program) dataset, in this dataset consist of data from early born baby's and their mother. It includes data of 747 patients from whom 25 covariates (x) where measured for a total of 100 treatments. [2],[1]

![Table 1](images/table_data.png)

This table shows the structure of the dataset. Adapted from [3].
The value t {1,0} indicates whether a patient is treated (1) or not (0), the y_factual is the outcome and the yc_factual is the outcome if one would have give the opposite treatment. This for example: if a diabetic patient received an insuline in treatment the t value would be 1 for the insulin treatment and the y_factual would be the "factual" outcome, the yc_factual outcome would then be the outcome given the opposite treatment, so when no insulin was given.

## CFR-net
For the recreating of the CFR network we firstly stepwise unraveled the Tensorflow code made by the authors. For this purpose we created a flowchart to gain insight in the proposed network:

![Flowchart](images/Flowdiagram.png)

It can be seen that the network consist of N layers (this is adjustable, but we choose for the default setting of 2 representation and 2 regression layers), the ReLU a non-linear activation, uses multiple dropout layers and at half-way it concatenates with the t-values. Then the user can choose for splitting the data set in two subsets: a subset with all data where t = 1 (treated group), and a subset for all data with t = 0 (not treated). If the user choose to do so then the datasets will be merged again to form 1 output.

In [1]:
import torch
import torch.nn as nn

split_output = False
class FCNet(nn.Module):
    """
    Simple fully connected neural network with residual connections in PyTorch.
    Layers are defined in __init__ and forward pass implemented in forward.
    """

    def __init__(self):
        super(FCNet, self).__init__()

        p = 0.3

        # Creating the linear layers
        self.h_in = nn.Linear(25, 100)
        self.layer_1 = nn.Linear(100, 100)
        self.layer_2 = nn.Linear(100, 100)
        self.layer_3 = nn.Linear(101, 100)
        self.layer_4 = nn.Linear(100, 100)
        self.layer_5 = nn.Linear(100, 100)

        # Creating the dropout layers
        self.do1 = torch.nn.Dropout(p=p)
        self.do2 = torch.nn.Dropout(p=p)
        self.do3 = torch.nn.Dropout(p=p)
        self.do4 = torch.nn.Dropout(p=p)
        self.do5 = torch.nn.Dropout(p=p)
        self.do6 = torch.nn.Dropout(p=p)

        # Creating the linear classifier layer
        self.fc6 = nn.Linear(100,1)

    # Forward pass for the first half of the network
    def forward(self, x, t):
        h = self.do1(F.relu(self.h_in(x)))
        h = self.do2(F.relu(self.layer_1(h)))
        h_rep = self.do3(F.relu(self.layer_2(h)))
        h = self._build_output_graph( h, t)
        self.h_rep = h_rep

        return h, h_rep

    # Forward pass for the second half of the network
    def _build_output (self, h):
        h = self.do4(F.relu(self.layer_3(h)))
        h = self.do5(F.relu(self.layer_4(h)))
        h = self.do6(F.relu(self.layer_5(h)))
        h = self.fc6(h)
        return h

    # Function for splitting data OR concatenate with t
    def _build_output_graph(self, h, t):
        t = torch.round(t)
        if split_output :
            i0 = torch.where(t < 1)
            i1 = torch.where(t > 0)

            temp=torch.index_select(h, 1, i0[1])
            rep0 = torch.cat((torch.index_select(h, 1, i0[1]),i0[0]),2)
            rep1 = torch.cat((torch.index_select(h, 1, i1[1]),i1[0]),2)

            y0 = self._build_output(rep0)
            y1 = self._build_output(rep1)

            y = dynamic_stitch([i0, i1], [y0, y1])
        else:
            h = torch.cat((h,t),1)
            y = self._build_output(h)

        return y

## Training loop
The training loop of the CFR network makes use of a ADAM optimizer and a MSE-loss criterion. In addition to the loss function there was also corrected for the distribution difference these corrections were added to the loss function. How this was done will be further described in the next section. Further we did not use any regularisation parameters, however the code posted by the authors gives different options for regularisation, but in the default settings it is not used.

## Corrections for distribution difference
### Sample re-weighting
To compensate for the difference in group size (treated / non-treated) a sample re-weighting was introduced. All the samples were re-weighted with the following formula [1]:

$wi = \frac{ti}{2u} + \frac{1-ti}{2(1-u)}$ for $i = 1 ... n $

With t {1,0} and u, the treatment prediction i.e. the chance of being treated based on the patients in this dataset. So for example if a patient has received a certain treatment let's say a patient had a surgery. Then the sample re-weighting would be 2 divided by two times the probability of being treated. If the probability of being treated is really high, close to 1, then a sample will get a relatively low weight. However, if the chance of being treated is very low then the sample would get a higher weight. The same could be observed for t = 0 (not treated). This will somewhat correct for the unequally distributed treatments groups i.e. it will correct for the fact that maybe a lot of patients received treatment A and only a few patients received treatment B.

In [2]:
    def sample_reweighting(t,p_t):
        w_t = t / (2 * p_t)             #p_t: chance of being treated
        w_c = (1 - t) / (2 * 1 - p_t)
        sample_weight = w_t + w_c
        return sample_weight

### Imbalance error
In addition to the re-weighting of the samples an imbalance error was introduced in the loss function. The imbalance error adjusts for the bias induced by the treatment group imbalance. There are different methods for the calculation of the imbalance error. For this project the squared linear Maximum Mean discrepancy (MMD) and the Wasserstein methods were used.

In [3]:
# Squared linear MMD imbalance error
def mmd2_lin(X,t,p):
    ''' Linear MMD '''

    it = torch.where(t>0)[0] # getting the positions
    ic = torch.where(t<1)[0]

    Xt = torch.index_select(X, 0, it) # Getting the nx100 for each value
    Xc = torch.index_select(X, 0, ic)

    mean_control = torch.mean(Xc,0) # mean of 1x100
    mean_treated = torch.mean(Xt,0)

    mmd = torch.sum(torch.square(2.0*p*mean_treated - 2.0*(1.0-p)*mean_control))

    return mmd

# Wasserstein imbalance error
def wasserstein(X,t,p,lam=10,its=10,sq=False,backpropT=False):
    """ Returns the Wasserstein distance between treatment groups """

    it = torch.where(t > 0)[0]  # getting the positions
    ic = torch.where(t < 1)[0]

    Xt = torch.index_select(X, 0, it)  # Getting the nx100 for each value
    Xc = torch.index_select(X, 0, ic)

    nc = Xc.shape[0]
    nt = Xt.shape[0]

    ''' Compute distance matrix'''
    if sq:
        M = pdist2sq(Xt,Xc)
    else:
        M = safe_sqrt(pdist2sq(Xt,Xc))

    ''' Estimate lambda and delta '''
    M_mean = torch.mean(M)
    M_drop = torch.nn.Dropout(10/(nc*nt))(M)
    delta = torch.max(M)
    eff_lam = lam/M_mean

    ''' Compute new distance matrix '''
    Mt = M
    row = delta*torch.ones(M.shape[1])
    col = torch.cat((delta*torch.ones(M.shape[0]),torch.zeros((1))),0)
    Mt = torch.cat((M, torch.unsqueeze(row, 0)), 0)
    Mt = torch.cat((Mt, torch.unsqueeze(col, 1)), 1)

    ''' Compute marginal vectors '''
    a = torch.cat((p * torch.ones((torch.where(t > 0)[0].shape[0],1)) / nt, (1 - p) * torch.ones((1,1))), 0)
    b = torch.cat(((1-p) * torch.ones((torch.where(t < 1)[0].shape[0],1)) / nc, p * torch.ones((1,1))), 0)

    ''' Compute kernel matrix'''
    Mlam = eff_lam*Mt
    K = torch.exp(-Mlam) + 1e-6 # added constant to avoid nan
    U = K*Mt
    ainvK = K/a

    u = a
    for i in range(0,its):
        u = 1.0/(torch.matmul(ainvK,( b / torch.transpose(torch.matmul(torch.transpose(u,0,1),K),0,1))))
    v = b/(torch.transpose(torch.matmul(torch.transpose(u,0,1),K),0,1))

    T = u*(torch.transpose(v,0,1)*K)

    E = T*Mt
    D = 2*torch.sum(E)

    return D, Mlam

The actual computations of these imbalance errors goes a bit beyond the scope of this blog post. However, it is good to know that there was corrected for the distribution imbalance in two different ways. This will also result in two different outcomes.

## Outcome measures
### PEHE
One of the main outcome measures of the project is the expected precision in Estimation of Heterogeneous Effect (PEHE). Which can be calculated using the following formula:

$ \epsilon PEHE = \int_x (\hat{\tau_f}(x)-\tau_f(x))^2 p(x) dx $

With:

$\hat{\tau_f}(x) = f(x,1) - f(x,0)$

Where $p(x)$ is the distribution of the data x, $f(x,1)$ is the hypothesis for a treatment given that the patient with data x is treated. And $f(x,0)$ is the hypothesis for a treatment given that the patient with data x is not treated. $\tau_f(x)$ is the ITE for this given treatment. Therefore, it can be concluded that the PEHE is a measure for the squared difference between hypothetical outcome for a treatment (calculated by the neural network) and the given outcome for the treatment. So it gives insight in how well the network can make predictions. For the actual outcome of this project is the square root of the PEHE used.

### ATE
The second main outcome is the absolute error of the average treatment effect (ATE). Following the paper this can be calculated using the following formula:

$\epsilon ATE = |\frac{1}{n}\sum\limits_{i=1}^{n} (f(x_i,1)-f(x_i,0)) - \frac{1}{n}\sum\limits_{i=1}^{n} (m_1(xi) - m_0(xi))|$

Rewriting knowing that $m_1(xi) - m_0(xi)$ is another description for the ITE gives:

$\epsilon ATE = |\frac{1}{n}\sum\limits_{i=1}^{n} \hat{\tau}_i - \frac{1}{n}\sum\limits_{i=1}^{n} \tau_i|$

The following code is just a short representation of the lines used for calculating the PEHE and the ATE.

In [4]:
import numpy as np

def pehe_ate(mu1, mu0, ycf_p,yf_p):
    # Calculation of PEHE
    eff = mu1-mu0 # ITE
    eff_pred = ycf_p - yf_p
    pehe = np.sqrt(np.mean(np.square(eff_pred-eff)))

    # Calulation of ATE
    ate_pred = np.mean(eff_pred)
    bias_ate = ate_pred-np.mean(eff)

    return pehe, bias_ate


## Results
After training our network we calculated the outcome measures (PEHE and ATE) for two different imbalance functions. By doing this we recreated the results of the orginal paper. The results are summarized in the following table:

<div>
    
<img src="images/results_table.png" width="500"/>
</div>

In addition to the outcome measures we also kept track of the losses during each iteration. These results can be summarized in a graph plotting the loss during training and the corresponding validation loss for every 100 iterations. We computed graphs for both the imbalance error functions.

<div>
<img src="images/output_graph_wass.png" width="500"/>
</div>
<div>
<img src="images/output_graph_mmd.png" width="500"/>
</div>


## Conclusion
Our presented results are in line with the results of the original paper. Keeping in mind that we were not aware of all the exact settings the authors used for the construction of their results. Additionally, about the imbalance error, it is hard to say which function performs the best because the results fluctuate very much. In the repository of our code the file ''main_cfrnet.py'' can be run to replicate our results.

We tried to reproduce the network based on the paper, however this had many complications given the amount of 'mathiness' and unclear explanation of the CFR network. To be able to reproduce the network we needed to carefully analyse the published code. This code had many complications with its outdated TensorFlow libraries and its vast amount parameters.

## Recommendations
The main reason of the difference in results between our reproduction and the original paper is probably because we did not know the network settings used for the creation of the results. These settings are for example: learning rate, dropout probability, regularisation etc. It would have been favorable to investigate in all the different options and selecting the best settings for the network. 
In addition, we were not able to split the data set in the network. It could be possible that splitting the dataset has an impact (positive or negative) on the results.
To try to test many other architectures of the CFR network, a variant of the project, in which the number of layers can be dynamically allocated, was implemented in the branch named Dynamic_network of our repository. This implementation (shown below) had no remarkable (it even went worse) effects in some of the tests performed when changing these values.

In [5]:
import torch
import torch.nn as nn

class FCNet(nn.Module):
    """
    Simple fully connected neural network with residual connections in PyTorch.
    Layers are defined in __init__ and forward pass implemented in forward.
    """

    def __init__(self, dim_in, n_in, dim_out, n_out, dropout_in, dropout_out):
        super(FCNet, self).__init__()

        self.in_layers=[]
        self.in_drop_layers = []
        self.out_layers = []
        self.out_drop_layers = []

        # Input layer
        self.h_in = nn.Linear(25, dim_in)
        self.do1 = torch.nn.Dropout(p=dropout_in)
        
        # Input group of layers
        for i in range(n_in):
            self.in_layers.append(nn.Linear(dim_in, dim_in))
            self.in_drop_layers.append(torch.nn.Dropout(p=dropout_in))

        # Add representation
        if not split_output:
            self.out_layers.append(nn.Linear(dim_out+1, dim_out))
        else:
            self.out_layers.append(nn.Linear(dim_out, dim_out))
        self.out_drop_layers.append(torch.nn.Dropout(p=dropout_out))

         # Output group of layers
        for i in range(n_out):
            self.out_layers.append(nn.Linear(dim_out, dim_out))
            self.out_drop_layers.append(torch.nn.Dropout(p=dropout_out))

        # Linear classifier layer
        self.h_out = nn.Linear(dim_out, 1)

## References
[1] Shalit, U. (2017). Estimating individual treatment effect: generalization bounds and algorithms.

[2] Information about the IHDP dataset can be found on: https://www.icpsr.umich.edu/web/HMCA/studies/9795

[3] Amit Sharma, Emre Kiciman, 2020, accessed on 23-03-2021, https://microsoft.github.io/dowhy/example_notebooks/dowhy_ihdp_data_example.html